In [ ]:
pip install sqlalchemy aiosqlite alembic

In [5]:
from sqlalchemy import create_engine, text, select, update, Table, Column, Integer, String, MetaData, insert, ForeignKey
from sqlalchemy.ext.asyncio import create_async_engine
from sqlalchemy.orm import Session, sessionmaker, DeclarativeBase, Mapped, mapped_column
from typing import Annotated
import datetime

In [6]:
sync_engine = create_engine(url="sqlite:///../data.db", echo=True, pool_size=5, max_overflow=10)
async_engine = create_async_engine(url="sqlite+aiosqlite:///../data.db", echo=True)

In [ ]:
with sync_engine.connect() as connect:
    result = connect.execute(text("SELECT * FROM transactions LIMIT 1"))
    print(f"{result.all()=}")

In [ ]:
async with async_engine.connect() as connect:
    result = await connect.execute(text("SELECT * FROM transactions LIMIT 1"))
    print(f"{result.all()=}")

In [8]:
sync_engine.dispose()
async_engine.dispose()

In [21]:

metadata_obj = MetaData()

currency_table = Table(
    "Currency",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("day", Integer),
)

def create_tables():
    metadata_obj.create_all(sync_engine)

def drop_tables():
    metadata_obj.drop_all(sync_engine)

def insert_data():
    with sync_engine.connect() as connect:
        #statement = text("INSERT INTO Currency (day) VALUES (1), (2);")
        statement = insert(currency_table).values([{"day": 3}, {"day": 4}, {"day": 5}])
        connect.execute(statement)
        connect.commit()

#drop_tables()
#create_tables()
#insert_data()

In [7]:

with Session(sync_engine) as session:
    pass

session_factory = sessionmaker(sync_engine)

with session_factory() as session:
    pass

str_200 = Annotated[str, 200]
class Base(DeclarativeBase):
    type_annotation_map = {
        str_200: String(200)
    }

class CurrencyORM(Base):
    __tablename__ = "Currency"
    id: Mapped[int] = mapped_column(primary_key=True)
    day: Mapped[int]

def insert_data():
    day99 = CurrencyORM(day=99)
    day16 = CurrencyORM(day=16)
    day17 = CurrencyORM(day=17)
    with session_factory() as session:
        session.add(day99)
        session.add_all([day16, day17])
        session.commit()

#insert_data()

In [18]:

intPK = Annotated[int, mapped_column(primary_key=True)]

class TransactionORM(Base):
    __tablename__ = "transactions"

    id: Mapped[intPK]
    date: Mapped[str]
    amount: Mapped[float]
    #day: Mapped[int] = mapped_column(ForeignKey(CurrencyORM.id))
    day: Mapped[int] = mapped_column(ForeignKey("Currency.id", ondelete="CASCADE"))
    #day: Mapped[int | None] = mapped_column(ForeignKey("Currency.id", ondelete="SET NULL"))
    datetime: Mapped[str] = mapped_column(server_default=text("TIMEZONE('utc', NOW())"))
    #datetime: Mapped[datetime.datetime] = mapped_column(default=datetime.datetime.utcnow())
    updated_at: Mapped[str] = mapped_column(server_default=text("TIMEZONE('utc', NOW())"))#, onupdate=datetime.datetime.utcnow)


def create_tables():
    Base.metadata.create_all(sync_engine)

create_tables()

2024-12-29 15:52:30,306 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-29 15:52:30,307 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Currency")
2024-12-29 15:52:30,309 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-29 15:52:30,310 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("transactions")
2024-12-29 15:52:30,310 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-29 15:52:30,310 INFO sqlalchemy.engine.Engine COMMIT


In [23]:

"""class TransactionORM(Base):
    __tablename__ = "transactions"

    transaction_id: Mapped[str] = mapped_column(primary_key=True)
    month: Mapped[str]
    datetime: Mapped[str]
    type: Mapped[str]
    account: Mapped[str]
    currency: Mapped[str]
    amount: Mapped[float]
    category: Mapped[str]
    point: Mapped[str]
    item: Mapped[str]
    comment: Mapped[str]"""

"""def select_currencies():
    with sync_engine.connect() as conn:
        query = select(currency_table)  # SELECT * FROM Currency
        result = conn.execute(query)
        days = result.all()
        print(f"{days=}")"""
"""
def update_currencies(day_id: int = 1, new_day: int = -1):
    with sync_engine.connect() as conn:
        #statement = text(f"UPDATE Currency SET day={new_day}") # forbidden, sql injection
        #statement = text("UPDATE Currency SET day=:new_day WHERE id=:day_id") # better
        #statement = statement.bindparams(new_day=new_day, day_id=day_id)
        #statement = update(currency_table).values(day=33).where(currency_table.c.id == day_id) 
        #statement = update(currency_table).values(day=33).filter(currency_table.c.id == day_id)
        statement = update(currency_table).values(day=new_day).filter_by(id=day_id)
        conn.execute(statement)
        conn.commit()
"""

def select_currencies():
    with session_factory() as session:
        # curr1 = session.get(CurrencyORM, 1)
        query = select(CurrencyORM)  # SELECT * FROM Currency
        result = session.execute(query)
        #days = result.all()
        days = result.first()
        print(f"{days=}")

def update_currencies(day_id: int = 1, new_day: int = -1):
    # with ORM first SELECT, then UPDATE, without ORM - just UPDATE
    with session_factory() as session:
        day_1 = session.get(CurrencyORM, day_id)
        day_1.day = new_day
        session.commit()


select_currencies()
update_currencies(new_day=12)
select_currencies()


2024-12-29 16:52:24,487 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-29 16:52:24,487 INFO sqlalchemy.engine.Engine SELECT "Currency".id, "Currency".day 
FROM "Currency"
2024-12-29 16:52:24,487 INFO sqlalchemy.engine.Engine [cached since 8.925s ago] ()
days=[(1, 322), (2, 2), (3, 3), (4, 4), (5, 5), (6, 99), (7, 16), (8, 17)]
2024-12-29 16:52:24,489 INFO sqlalchemy.engine.Engine ROLLBACK
2024-12-29 16:52:24,491 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-29 16:52:24,492 INFO sqlalchemy.engine.Engine SELECT "Currency".id AS "Currency_id", "Currency".day AS "Currency_day" 
FROM "Currency" 
WHERE "Currency".id = ?
2024-12-29 16:52:24,492 INFO sqlalchemy.engine.Engine [cached since 935.7s ago] (1,)
2024-12-29 16:52:24,492 INFO sqlalchemy.engine.Engine UPDATE "Currency" SET day=? WHERE "Currency".id = ?
2024-12-29 16:52:24,492 INFO sqlalchemy.engine.Engine [cached since 124.3s ago] (12, 1)
2024-12-29 16:52:24,498 INFO sqlalchemy.engine.Engine COMMIT
2024-12-29 16:52:24,5